# 方案说明
本方案是一个简单的根据0基础入门的示例中的识别手写数字改写而成的 常规赛：PALM眼底彩照视盘探测与分割 程序。<br />
本方案直接进行分割，不进行图片分类，因为按照理论来说，如果眼睛正常，并且神经网络学习到了所有的信息，则会将正常眼球照片分割为整块的白色，仅对病变部分分割出黑色。但实际上，这种认知是假设性的，因为神经网络只是有机会学到这个信息，有多大概率学到这个内容是完全不受保证的，所以对于实际问题，我建议务必考虑将分类和分割联系在一起做，而不是用分割程序简单代替分类。<br />
本方案得分0.64594，DICE为0.42351，F1值为0.97959。

本方案从头到尾直接运行即可，整体布局为 **数据获取 -> 构造模型和数据读取器 -> 训练 -> 预测 -> 平滑处理和打包**。<br />

## 具体实现策略<br />
&ensp;&ensp;1. 依次读取训练图片，将图片转为灰度模式，将图片缩放到指定尺寸。<br />
&ensp;&ensp;2. 构造神经网络和数据读取器，这部分直接参考0基础入门的示例中的识别手写数字。<br />
&ensp;&ensp;3. 训练数据，通过文件名分批次读取数据进行训练。<br />
&ensp;&ensp;4. 预测数据，通过文件名分批次读取数据进行预测。<br />
&ensp;&ensp;5. 预测后的结果通常来说并不能完美地划分，总是包含很多噪点（比如说对于大部分应当处于黑色部分的像素点是黑色，但是仍有一部分像素点是白色的），因此通过一个卷积层进行平滑操作。最后将平滑后的结果缩放到原图片大小，打包。

# 数据准备

In [ ]:
# 刷新内存资源
! rm -rf ./work/
! mkdir ./work/

In [ ]:
# 下载数据与解压数据
url='https://bj.bcebos.com/v1/dataset-bj/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/%E5%B8%B8%E8%A7%84%E8%B5%9B%EF%BC%9APALM%E7%9C%BC%E5%BA%95%E5%BD%A9%E7%85%A7%E8%A7%86%E7%9B%98%E6%8E%A2%E6%B5%8B%E4%B8%8E%E5%88%86%E5%89%B2.zip'

if not os.path.exists('./work/Train_and_test.zip'):
    print("Downloading start!")
    urllib.request.urlretrieve(url, "./work/Train_and_test.zip")  
    print("Downloading end!")
else:
    print("Already Downloading")

! unzip ./work/Train_and_test.zip -d ./work

# 程序开始
## 引入包

自定义变量包括:<br />
&ensp;&ensp;image_size 		 # 缩放图片的大小<br />
&ensp;&ensp;batchsize	   	# 每次读取数据的大小<br />
&ensp;&ensp;epoch_num		 # 训练批次<br />

In [3]:
# 引入包以及全局变量声明
import paddle
import paddle.nn.functional as F
import os
import numpy as np
from paddle.nn import Conv2D, MaxPool2D, Linear
import pandas as pd
from PIL import Image
import sys 
import random
import urllib 
import requests   
from paddle.nn.initializer import Assign

global image_size;
global batchsize;

image_size=64;
batchsize=100;
epoch_num=3;

## 图片分割
### 构造神经网络并定义数据读取器

In [83]:
# 定义一个三层全连接层组成的网络
class Mymodel(paddle.nn.Layer):
    def __init__(self):
        super(Mymodel, self).__init__()
        global image_size
        self.fc = Linear(in_features=image_size*image_size, out_features=image_size*image_size*4)
        self.fc2 = Linear(in_features=image_size*image_size*4, out_features=image_size*image_size*4)
        self.fc3 = Linear(in_features=image_size*image_size*4, out_features=image_size*image_size)

    # 定义网络前向计算过程，卷积后紧接着使用池化层，最后使用全连接层计算最终输出
    def forward(self, inputs):
        x = self.fc(inputs);
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x);
        x = F.sigmoid(x)
        return x

# 定义数据读取器
def load_data(mylist,mode='train'):
    global image_size
    global batchsize
    # 默认使用灰度图像
    input_channel=1
    
    # 根据需要指定对应的文件目录
    if mode == 'train':
        path='work/常规赛：PALM眼底彩照视盘探测与分割/Train/fundus_image/'
    elif mode=='test':
        path='work/常规赛：PALM眼底彩照视盘探测与分割/PALM-Testing400-Images/'
    else:
        raise Exception("mode can only be one of ['train', 'test']")
    
    # 获得数据集长度
    data_length = len(mylist)
    
    # 定义数据集每个数据的序号，根据序号读取数据
    index_list = list(range(data_length))
    
    # 定义数据生成器
    def data_generator():
        if mode == 'train':
            # 训练模式下打乱数据
            random.shuffle(index_list)
        imgs_list = []
        labels_list = []
        for i in index_list:
            # 将数据处理成希望的类型，即处理成image_size*image_size的向量
            im = Image.open(path + mylist[i] + '.jpg')
            im = im.resize((image_size, image_size), Image.ANTIALIAS)
            if input_channel==1:
                im = im.convert('L')
                img = np.array(im).reshape(1, -1).astype(np.float32)/255
            imgs_list.append(img.tolist()[0]) 

            # 读取标签
            if mode == 'train':
                im = Image.open('work/常规赛：PALM眼底彩照视盘探测与分割/Train/Disc_Masks/' + mylist[i] + '.bmp')
                im = im.resize((image_size, image_size), Image.ANTIALIAS)
                label = np.array(im).astype(np.int64)/255
                label = label.astype(np.int64)
                label = label.reshape(1, -1)
                label = label.tolist()[0]
            else: label=[]
            labels_list.append(label)
            if len(imgs_list) == batchsize:
                # 获得一个batchsize的数据，并返回
                yield np.array(imgs_list), np.array(labels_list)
                # 清空数据读取列表
                imgs_list = []
                labels_list = []
    
        # 如果剩余数据的数目小于BATCHSIZE，
        # 则剩余数据一起构成一个大小为len(imgs_list)的mini-batch
        if len(imgs_list) > 0:
            yield np.array(imgs_list), np.array(labels_list)
    return data_generator

### 获得数据列表

In [84]:
name = [name for name in os.listdir('work/常规赛：PALM眼底彩照视盘探测与分割/Train/fundus_image') if name.endswith('.jpg')]

train_name_list=[]
for i in name:
    tmp = os.path.splitext(i)
    train_name_list.append(tmp[0])

name = [name for name in os.listdir('work/常规赛：PALM眼底彩照视盘探测与分割/PALM-Testing400-Images') if name.endswith('.jpg')]

test_name_list=[]
for i in name:
    tmp = os.path.splitext(i)
    test_name_list.append(tmp[0])

### 训练网络

In [85]:
# 构建模型
model=Mymodel()

model.train()

# 构建用于分批读取数据训练的数据读取器
train_loader = load_data(train_name_list,mode='train')

opt = paddle.optimizer.Adam(learning_rate=0.001, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),parameters=model.parameters())

print('train start')

# 正式训练
for epoch_id in range(epoch_num):
    for batch_id, data in enumerate(train_loader()):
        images, labels = data
        images = paddle.to_tensor(images)
        labels = paddle.to_tensor(labels) 
        images = images.astype(dtype='float32')
        labels = labels.astype(dtype='float32')
        
        predicts = model(images)
        loss = F.square_error_cost(predicts, labels)
        avg_loss = paddle.mean(loss)

        if 1:
            print("{} epoch {} batch: los is {}".format(epoch_id,batch_id,avg_loss.numpy()))
        
        #后向传播，更新参数的过程
        avg_loss.backward()
        opt.step()
        opt.clear_grad()

print('train finish')

paddle.save(model.state_dict(), 'Mymodel')

### 预测

In [87]:
model=Mymodel()
param_dict = paddle.load('./Mymodel')
model.load_dict(param_dict)

test_loader = load_data(test_name_list,mode='test')

print('test start')

# 进行预测，将结果保存在mylist里
mylist=[]
for batch_id, data in enumerate(test_loader()):
        print(batch_id)
        images, labels = data
        images = paddle.to_tensor(images)
        images = images.astype(dtype='float32')
        
        predicts = model(images)
        mylist=mylist+list(predicts.numpy())

print('test finish')

### 打包结果

In [101]:
!mkdir './Disc_Segmentation/'

# 定义卷积让结果变得平滑一点
c_size=5
w=np.ones([c_size,c_size]).astype(dtype='float32')/(c_size*c_size)
w = w.reshape([1, 1, c_size, c_size])
conv = Conv2D(in_channels=1, out_channels=1, kernel_size=[c_size, c_size],weight_attr=paddle.ParamAttr(initializer=Assign(value=w)),padding=int((c_size-1)/2),padding_mode='reflect')

for i in range(len(test_name_list)):
    if i%50==0:
        print(i,end=" ")
        print(len(test_name_list))
    # 读取原图片的尺寸，用于缩放预测结果到原尺寸
    picdir='work/常规赛：PALM眼底彩照视盘探测与分割/PALM-Testing400-Images/' + test_name_list[i] + '.jpg'
    im = Image.open(picdir)
    width=np.array(im).shape[1]
    height=np.array(im).shape[0]
    
    picdir='./Disc_Segmentation/' + test_name_list[i] + '.png'
    tmp=mylist[i].reshape(image_size,image_size).astype('int')
    
    # 平滑处理
    x = paddle.to_tensor(tmp,dtype='float32')
    x = paddle.reshape(x,[1, 1, image_size, image_size])
    y = conv(x)
    tmp = y.numpy().reshape(image_size,image_size).round()*255

    img = Image.fromarray(tmp.astype('uint8'))
    im = img.resize((width, height), Image.ANTIALIAS)
    im.save(picdir)

# 打包结果
! zip -q -r result.zip ./Disc_Segmentation/

# 总结和一些讨论

1. 根据结果可以看到，这种全连接的架构还是能够学习到一些信息的，主要是圆形瞳孔的信息，但并不代表学习到的内容是所需要的划分目标，有时候也会将其他的圆形赋值黑色像素。<br />
2. 本程序之所以能够分数不错，主要是测试集大多都是需要划分的，很少有阴性数据。<br />
3. 最后的平滑处理虽然能够对得分有一些改善，但是改善是非常有限的，所以还是应该从基础架构入手。<br />
4. 由于上个月我被NLP吸引过去了，所以这个问题没有进行非常深入的了解，虽然这个模型较为简朴，但是也还可以进行一些优化，比如在网络内增加卷积层，也许会有一些效果的提升。<br />

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 